In [1]:
import torch
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('../input/copper_returns_5d_final.csv')
features = data.drop(columns=['Date', 'LMCADY_acu_5d', 'LMCADY_std_5d'])  # Asegúrate de excluir también la variable objetivo de los features
target = data['LMCADY_acu_5d']

In [3]:
features

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG
0,0.001241,0.001047,-0.000767,0.002808,0.006127,0.001182,-0.003199,0.009327
1,-0.000733,-0.000182,0.002015,0.003064,-0.005829,0.002399,-0.002942,-0.000409
2,0.009474,0.009322,-0.004022,0.001404,0.007491,0.002585,0.007338,0.002853
3,-0.002514,-0.002130,-0.005961,-0.003226,-0.001953,-0.006740,0.002320,0.033895
4,0.005432,0.005893,0.004159,-0.008826,0.003441,-0.006092,-0.013172,-0.026096
...,...,...,...,...,...,...,...,...
5534,-0.009774,-0.009724,0.006872,0.014287,-0.004085,0.014765,-0.018041,-0.010309
5535,0.004627,0.004489,-0.005850,0.006588,0.002701,0.008892,0.011519,0.020312
5536,-0.010746,-0.010210,0.004903,-0.021980,0.016248,-0.018378,0.000000,-0.001531
5537,-0.013035,-0.013388,0.004977,0.000484,-0.014186,-0.003775,-0.014804,-0.014724


In [4]:
scaler = StandardScaler()
features = scaler.fit_transform(features)
features

array([[ 8.88174336e-02,  7.44158059e-02, -1.61553556e-01, ...,
         1.40393504e-01, -2.72839869e-01,  4.81635337e-01],
       [-3.41047918e-02, -2.12838525e-04,  4.18921718e-01, ...,
         2.63621263e-01, -2.48888913e-01, -1.54865105e-02],
       [ 6.01515341e-01,  5.77035466e-01, -8.40686743e-01, ...,
         2.82472734e-01,  7.09955572e-01,  1.51088802e-01],
       ...,
       [-6.57712049e-01, -6.09357555e-01,  1.02160791e+00, ...,
        -1.83950901e+00,  2.55342740e-02, -7.27844599e-02],
       [-8.00281396e-01, -8.02392606e-01,  1.03697816e+00, ...,
        -3.61342573e-01, -1.35527210e+00, -7.46369564e-01],
       [-1.16351169e+00, -1.18211667e+00, -2.23029805e+00, ...,
        -2.30137329e+00,  5.64594036e-01, -3.97293980e-01]])

In [5]:
# mostrar desviacion estandar de target
target.std()

0.034782023369004565

In [6]:
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [10]:
train_dataset   = TensorDataset(torch.tensor(X_train, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.float))
val_dataset     = TensorDataset(torch.tensor(X_val, dtype=torch.float), torch.tensor(y_val.values, dtype=torch.float))
train_loader    = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader      = DataLoader(val_dataset, batch_size=64)

In [8]:
features.shape

(5539, 8)

In [11]:
class TransformerModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=features.shape[1], nhead=4, dim_feedforward=128)
        self.encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=1)
        self.regressor = nn.Linear(features.shape[1], 1)
        
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.regressor(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [12]:
model = TransformerModel()
trainer = pl.Trainer(max_epochs=50)
trainer.fit(model, train_loader, val_loader)

c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: c:\Users\gapuj\repositories\GitHub\tesis\entrenami

c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
C:\Users\gapuj\AppData\Local\Temp\ipykernel_7224\3376634985.py:22: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)
c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\pytor

Epoch 0:   0%|          | 0/70 [00:00<?, ?it/s] 

C:\Users\gapuj\AppData\Local\Temp\ipykernel_7224\3376634985.py:16: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


Epoch 0: 100%|██████████| 70/70 [00:00<00:00, 118.51it/s, v_num=0]

...

Epoch 0: 100%|██████████| 70/70 [00:00<00:00, 100.19it/s, v_num=0]

C:\Users\gapuj\AppData\Local\Temp\ipykernel_7224\3376634985.py:22: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


Epoch 49: 100%|██████████| 70/70 [00:00<00:00, 129.36it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 70/70 [00:00<00:00, 76.01it/s, v_num=0] 


In [13]:
y_val_pred = [model(torch.tensor([x], dtype=torch.float)).item() for x in X_val]
final_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"RMSE final en el conjunto de validación: {final_rmse}")
print("Stdev de valores reales: {}".format(y_val.std()))

C:\Users\gapuj\AppData\Local\Temp\ipykernel_7224\1503390093.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  y_val_pred = [model(torch.tensor([x], dtype=torch.float)).item() for x in X_val]


RMSE final en el conjunto de validación: 0.03637988260735615
Stdev de valores reales: 0.034027450312969415
